In [14]:
import joblib
import os
import pandas as pd
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
from itertools import combinations
%matplotlib inline

In [15]:
pd.options.mode.chained_assignment = None 

In [16]:
HERE = os.path.dirname(os.path.abspath('__file__'))
DATA = os.path.join(HERE, 'Data')

SSGSEA_LIHC_KEGG = os.path.join(
    DATA,
    "kegg_lihc.tsv"
)

GENE_SETS_KEGG = os.path.join(
    DATA,
    "kegg_geneset_final.gmt"
)

TRAINED_MODEL = os.path.join(
    DATA,
    "lihc-kegg_0.joblib"
)

LIHC_LABELS = os.path.join(
    DATA,
    "phenotype_classes_lihc.cls"
)
DRUGBANK_TO_GENES_ID = os.path.join(
    DATA,
    "drugbank_to_genes_ID.csv"
)
HGNC_ID_MAP_TO_GENE_NAME = os.path.join(
    DATA,
    "HGNC_ID_map_to_gene_name.csv"
)

APPROVED_DRUG_LIVER= os.path.join(
    DATA,
    "approved_drug_liver.csv"
)

In [17]:
## Lable preparation

lihc_labels = pd.read_csv(LIHC_LABELS, sep = "\t")
lihc_labels.drop(lihc_labels.index[0], inplace=True)
lihc_labels = lihc_labels.rename(index={1:'label'})
lihc_labels = lihc_labels.transpose()

temp_lable = []

for lable in lihc_labels.label:
    temp_lable = lable.split(' ')

In [18]:
## Patient_Pathway dataframe preparation

# Transpose the dataframe's columns and rows
raw_data = pd.read_csv(SSGSEA_LIHC_KEGG, sep = "\t", index_col=0).transpose()

# Append the data lable as a column to main dataframe
raw_data.insert(311, "label", temp_lable, True)

# Convert the data lable into numerical value
num_labels = {"Normal": 0, "Tumor": 1} 
raw_data.label = [num_labels[item] for item in raw_data.label]

# # Removing the index column 
raw_data.reset_index(drop=True, inplace=True)

In [19]:
def parse_gmt_file(gmt_path: str, min_size=3, max_size=3000):
    """Parse gmt file."""
    with open(gmt_path) as f:
        genesets_dict = {
            line.strip().split("\t")[0]: line.strip().split("\t")[2:]
            for line in f
        }
    return genesets_dict

In [20]:
pathway_genes_dict = parse_gmt_file(GENE_SETS_KEGG)

In [21]:
## Read the drugbank_to_genes file and filter all source_databases but drugbank

drugbank_to_genes_ID = pd.read_csv(DRUGBANK_TO_GENES_ID,sep = "\t")
drugbank_to_genes_ID_keep_drugbank = drugbank_to_genes_ID.loc[drugbank_to_genes_ID['source_database'] == "drugbank"]

for i in range(len(drugbank_to_genes_ID_keep_drugbank["source"])):
    temp_drug = drugbank_to_genes_ID_keep_drugbank["source"].iloc[i]
    temp_drug = temp_drug.split(':')[1]
    drugbank_to_genes_ID_keep_drugbank["source"].iloc[i] = temp_drug

In [22]:
## Read the HGNC_ID to gene_name mapped file downloaded from HGNC website 

HGNC_ID_map_to_gene_name = pd.read_csv(HGNC_ID_MAP_TO_GENE_NAME, sep = "\t")

In [23]:
## Drug Dataset Preparation

clinical_trials_drugs = pd.read_csv('https://raw.githubusercontent.com/drug2ways/results/master/validation/data/DrugBank-MeSH-slim-counts.tsv',sep = "\t")
clinical_trials_drugs = clinical_trials_drugs.loc[clinical_trials_drugs['condition'] == "D008113"]
approved_liver_drugs = pd.read_csv(APPROVED_DRUG_LIVER,sep = "\t")

In [24]:
## Statistics

temp_intersect_drugbank_approved_lihc = []
temp_intersect_drugbank_clinical_trials_lihc = []

for drug in drugbank_to_genes_ID_keep_drugbank["source"].values:
    if drug in clinical_trials_drugs["drugbank_id"].values:
        temp_intersect_drugbank_clinical_trials_lihc.append(drug)
    if drug in approved_liver_drugs["Approved_drug_liver"].values:
        temp_intersect_drugbank_approved_lihc.append(drug)
        
intersect_drugbank_clinical_trials_lihc = set(temp_intersect_drugbank_clinical_trials_lihc)
#print(len(intersect_drugbank_clinical_trials_lihc))
intersect_drugbank_approved_lihc = set(temp_intersect_drugbank_approved_lihc)
#print(len(intersect_drugbank_approved_lihc))

In [25]:
## Load the trained classifier

trained_model = joblib.load(open(TRAINED_MODEL, "rb"))

/home/skhatami/miniconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator ElasticNetCV from version 0.20.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [26]:
## Replace gene_id with gene_name in drugbunk gene target file

for gene_ID in drugbank_to_genes_ID_keep_drugbank["target"]:
    for gene_id in HGNC_ID_map_to_gene_name["HGNC ID"]:
        
        # Skip genes that are not the same
        if gene_ID != gene_id:
            continue
            
        row_index_HGNC_ID_map = HGNC_ID_map_to_gene_name[HGNC_ID_map_to_gene_name["HGNC ID"] == gene_id].index.values[0]
        gene_symbol = HGNC_ID_map_to_gene_name.iloc[row_index_HGNC_ID_map, HGNC_ID_map_to_gene_name.columns.get_loc('Approved symbol')]
        drugbank_to_genes_ID_keep_drugbank['target'] = drugbank_to_genes_ID_keep_drugbank['target'].replace(gene_ID,gene_symbol)

In [27]:
## Drug dataframe prepration for calculating score of a pathway including all of its involving genes 

# Drop the source_database column as all drugs in dataframe are coming from drugbank
drugbank = drugbank_to_genes_ID_keep_drugbank.drop('source_database', 1)

# Group the targeting genes based on the drugs
drugbank_groupby_drug = drugbank.groupby('source')

# Forming a list of unique drugs used further for preparation of dataframe containing drugs and its targeted pathway 
## and all targeted genes involved in that pathway
unique_drug = drugbank["source"].unique()

In [28]:
## Preparing dataframe containing drugs and its targeted pathway and all targeted genes involved in that pathway
## plus its corresponding affecting score

# creating an empty dataframe
pathway_to_score = pd.DataFrame(columns=['drug_ID','pathway','affection_rate','gene_name'])

for drug in range(len(unique_drug)):
    
    # get the subset of drugbank dataset with regards to the a data
    temp_drug_gene_relation_df = drugbank_groupby_drug.get_group(unique_drug[drug])
    
    # drop the drug column to turn it to dict for efficient looping
    temp_drug_gene_relation_df = temp_drug_gene_relation_df.drop("source",1)
    
    # convert the subset dataframe to dictionary
    temp_gene_score_dict = dict(temp_drug_gene_relation_df.values.tolist())   
    
    # loop over pathway_genes_dict genes and pathways
    for pathways, genes in pathway_genes_dict.items():
        temp_gene= genes
        temp_pathway = pathways
        
        # loop over subset dataframe converted dict genes and scores
        for gene, score in temp_gene_score_dict.items():
            gene_temp = gene
            score_temp = score
            
            # find all genes of a pathway and makeing a dataframe out of that with all details (drug,gene,pathway,affecting score)
            if gene_temp in temp_gene:
                pathway_to_score = pathway_to_score.append({'drug_ID':unique_drug[drug],'pathway': temp_pathway, 'affection_rate': score_temp, 'gene_name': gene_temp}, 
                                                       ignore_index=True)

In [29]:
## Preparing dataframe with a score per drug per pathway(considering all of its involving genes)

# creating an empty dataframe
pathway_scores = pd.DataFrame(columns=['drug_ID','Pathway', 'Finall_affected_score'])

# Groupby the last step provided dataframe by 'drug_ID','pathway'
pathway_to_score_groupby = pathway_to_score.groupby(['drug_ID','pathway'])

for drug,path,score,gene in pathway_to_score.values:
    
    # get the subset of last step prepared dataframe with regards to the drug and correponding pathway (considering all of its involving genes)
    temp_pathway_to_score_df = pathway_to_score_groupby.get_group((drug,path))
    
    # calculating the sum of the scores for all the genes of a pathway
    temp_affected_score = temp_pathway_to_score_df['affection_rate'].sum()
    
    # calculating the mean 
    finall_affected_score = temp_affected_score / (temp_pathway_to_score_df.shape[0])
    
    # make a dataframe dataframe with a score per drug per pathway
    pathway_scores = pathway_scores.append({'drug_ID':drug,'Pathway': path, 'Finall_affected_score': finall_affected_score},ignore_index=True)

# Drop the duplicate if there is any
pathway_scores.drop_duplicates(subset=['drug_ID','Pathway'],keep="first",inplace=True) 

In [ ]:
pathway_scores.head()

In [ ]:
## Splite samples based on our desired lables

def splite_samples(raw_data, desired_label):
    
    # Split the subset of pateints having desired lable
    desired_label_sample= raw_data.loc[raw_data['label'] == desired_label]
    
    # Dataframe including the other subset of patients with undisred lable
    undesired_label_sample = pd.concat([raw_data, desired_label_sample]).drop_duplicates(keep=False)
    
    return desired_label_sample, undesired_label_sample

In [ ]:
## Modify the pathway score of each patient with regards to each drug available in drugbank

def path_score_modification(drug_name, raw_data, desired_label):
     
    desired_path_score_changed_sample, undesired_path_score_changed_sample = splite_samples(raw_data, desired_label)
    
    # Get subset of dataframe with a score per drug per pathway with regards to selected drug
    temp_pathway_drug_all_gene_score = pathway_scores.groupby('drug_ID')
    pathway_drug_including_all_gene_score = temp_pathway_drug_all_gene_score.get_group(drug_name)
    
    # Dictionary of pathways affected by the drug to their respective scores
    affected_pathway_to_score = {
        pathway: score
        for _, pathway, score in pathway_drug_including_all_gene_score.values
    }
                 
    # For each sample id
    for sample in range(len(desired_path_score_changed_sample)):
            
        # For each pathway that we have to modify a score in all patients since it is targetted by the drug
        for pathway in affected_pathway_to_score:
                        
            # Get related affection scored calculated per drug per pathway
            affection_score = affected_pathway_to_score[pathway]
            
            pathway_column = desired_path_score_changed_sample.columns.get_loc(pathway)

            # Get current pathway score of each patient per pathway
            current_score = desired_path_score_changed_sample.iloc[sample, pathway_column]

            # Start point of main scoring function
            # The range of scores calculated for the pathways per drugs is between [1 and -1],
            # The range was broken down to intervals [-1,-0.5],[0.5,0],[0],[0,0.5] and [0.5,1]

            if affection_score >= 0.5:
                if current_score < 0:
                    desired_path_score_changed_sample.iloc[sample, pathway_column] = 0
                else:
                    desired_path_score_changed_sample.iloc[sample, pathway_column] = 1

            elif 0 <= affection_score < 0.5:
                if current_score < 0:
                    desired_path_score_changed_sample.iloc[sample, pathway_column] = 0

                else:
                    desired_path_score_changed_sample.iloc[sample, pathway_column] = 0.5

            elif affection_score == 0:
                    desired_path_score_changed_sample.iloc[sample, pathway_column] = current_score

            elif -0.5 <= affection_score < 0:
                if current_score < 0:
                    desired_path_score_changed_sample.iloc[sample, pathway_column] = -0.5
                else:
                    desired_path_score_changed_sample.iloc[sample, pathway_column] = 0

            elif affection_score <= -0.5:
                if current_score < 0:
                    desired_path_score_changed_sample.iloc[sample, pathway_column] = -1
                else:
                    desired_path_score_changed_sample.iloc[sample, pathway_column] = 0

    modified_raw_data_frame = pd.concat([undesired_path_score_changed_sample, desired_path_score_changed_sample], axis=0)
    
    return modified_raw_data_frame

In [ ]:
top_auc_drug_df = pd.read_csv('/home/skhatami/Projects/drug-pathway-revert/drug-pathways-revert/ssGSEA_Results/pathway_drug_score_lihc_df_test.csv', sep = "\t")
top_auc_drug = []

for drug, auc in top_auc_drug_df.values:
    if auc <= 0.5:
        top_auc_drug.append(drug)


pathway_scores_groupby_drug_ID = pathway_scores.groupby('drug_ID')

temp_df_drug_comb = pd.DataFrame()

for drug in top_auc_drug:
    pathway_drug_including_all_gene_score = pathway_scores_groupby_drug_ID.get_group(drug)
    temp_df_drug_comb = temp_df_drug_comb.append(pathway_drug_including_all_gene_score)
    
drug_comb_list = list(combinations(top_auc_drug, 2)) 

In [ ]:
def auc_per_drug(drug_data_set,model,data,desired_lable, raw_data_set_lable):
    
    pathway_drug_score_lihc = pd.DataFrame(columns=['drug_1', 'drug_2', 'ROC_SCORE'])

    for lst in tqdm(drug_comb_list):
        d_1 =  lst[0]
        d_2 =  lst[1]
        temp_data_set_d_1 = path_score_modification(d_1,raw_data,desired_lable)
        temp_data_set_d_2 = path_score_modification(d_2,temp_data_set_d_1,desired_lable)
        prepared_data_set_for_prediction = temp_data_set_d_2.iloc[:,:311]
        prediction = trained_model.predict(prepared_data_set_for_prediction)
        ROC_SCORE = roc_auc_score(raw_data.label,prediction)
        pathway_drug_score_lihc = pathway_drug_score_lihc.append({'drug_1': d_1, 'drug_2': d_2,'ROC_SCORE': ROC_SCORE},ignore_index=True)
        print(d_1,d_2,ROC_SCORE)
    return pathway_drug_score_lihc              

In [ ]:
top_auc_drug_comb_df = auc_per_drug(drug_comb_list,trained_model,raw_data,1,raw_data.label)